<a href="https://colab.research.google.com/github/KlaraDewitte/Machine_Learning_course_UGent_D012554_kaggle/blob/master/Poging2SVM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [0]:
trainset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_train.csv")

testset = pd.read_csv("https://raw.githubusercontent.com/sdgroeve/Machine_Learning_course_UGent_D012554_kaggle/master/eeg_test.csv")

In [0]:
labels = trainset.pop('label')
indices = testset.pop('index')

In [0]:
from sklearn.preprocessing import StandardScaler

Scaler = StandardScaler()
Scaler.fit(trainset)

trainset_scaled = pd.DataFrame(Scaler.transform(trainset))

testset_scaled= pd.DataFrame(Scaler.transform(testset))



In [0]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.model_selection import cross_val_score

In [0]:
X = trainset_scaled
y = labels

In [10]:
OptiC = 0
Possibilities = ['linear', 'poly', 'rbf', 'sigmoid']
search_space = [0.01,0.1,1,10,100, 1000]
params = dict(C=search_space)

for pos in Possibilities:
  print(pos)

  clf=SVC(kernel=pos)
  grid_search = GridSearchCV(clf, param_grid=params)
  grid_search.fit(X, y)

  OptiC=grid_search.best_estimator_.C
  print('Optimale C =' + str(OptiC))

  clf=SVC(OptiC, kernel=pos)
  clf.fit(X,y)

  score_auc = np.mean(cross_val_score(clf,X,y,cv=10,scoring='roc_auc'))

  print('auc=' + str(score_auc))


linear
Optimale C =1
auc=0.6744776630097732
poly
Optimale C =1000
auc=0.9142904119326138
rbf
Optimale C =100
auc=0.9605496725496728
sigmoid
Optimale C =0.01
auc=0.6115343209288163


In [9]:
clf=SVC(C=100, kernel='rbf', probability=True)
clf.fit(X,y)
predictions = clf.predict_proba(testset_scaled)[:,1]
print(predictions)

[7.13874942e-06 7.72867001e-01 4.89700430e-03 ... 9.95681929e-01
 7.18506648e-01 3.22125870e-02]


In [11]:
dfpredictions= pd.DataFrame(({'index': indices, 'label':predictions}))
print(dfpredictions)

       index     label
0          0  0.000007
1          1  0.772867
2          2  0.004897
3          3  0.999993
4          4  1.000000
...      ...       ...
12887  12887  0.091558
12888  12888  0.999994
12889  12889  0.995682
12890  12890  0.718507
12891  12891  0.032213

[12892 rows x 2 columns]


In [0]:
filename = "my_prediction_results_Poging2SVM.csv"

#make sure to not write the Pandas index column (index=False)
dfpredictions.to_csv(filename,index=False)